# Prepare the Data for Modeling and feature importance visual

## Feature Selection

In this section, I will select specific features for model training. The original dataset contains 265 features, most of which are categorical attributes, dominantly the list of drug names that are approved as pain medications that need special AED licensed prescriber.Here,my main target is to predict the overdoses using the  the list of prescriber specialists with NPI information and the FDA approved drug lists. The data set has only total opiod cliams and total claim counts and i took the standard prescription for opiod drugs is 84 days which is 12 weeks. Therefore, an additional feature was created in the data cleaning, pre processing that shows avg_op_supply which i limit to the standard to be 84 and any ones who prescribes the category of medications under the controlled substance act(CSA). The CSA is used for organizing drugs based on the risk of abuse or harm to users in five and my capestone focuses on the approved by [FDA](https://www.fda.gov/drugs/drugsafety/informationbydrugclass/ucm251735.htm) and [DEA](https://www.deadiversion.usdoj.gov/schedules/) to be prescribed by licensed and certified health professional.  These drug lists that are considered controlled substances under the Controlled Substances Act (CSA) are divided into five schedules.  An updated and complete list of the schedules is published annually in [Title 21 Code of Federal Regulations (C.F.R.) §§ 1308.11 through 1308.15](https://www.deadiversion.usdoj.gov/21cfr/cfr/2108cfrt.htm).

Therfore, for analysis and visualization  i only took those physicians who have greater opood cliams from the center for medical services dataset and created a feature average opiod supply  by dividing the total cliam by the opiod cliam and reviewed those specialists who prescribe more than the set limit of 12 weeks and that also become a new feature that measures the length of opiod prescription supply more than 84 days. As been visualized in notebook 1.3, significant amount of prescribers are family physicians, followed by internal medicine,nurse practitioners, Physicain Assistant and physical medicine and rehabilitation. When i group based on speciality and the vsuals can be accessed   [here top 20](https://plot.ly/~kiros/112) and the lists are as follows:-
  > $'Cardiology', 'Nephrology', 'Endocrinology', 'Neurology'  'General Practice', 'Internal Medicine', 'Urology'$,   > $'Pulmonary Disease', 'Gastroenterology', 'Family Practice', 'Infectious Disease','Psychiatry'$ 
  >$'Nurse Practitioner', 'Ophthalmology','Allergy/Immunology', 'Otolaryngology', 'Psychiatry & Neurology'$ , $'Rheumatology', 'Dermatology', 'Obstetrics/Gynecology'$

In [1]:
# importing necessary libraries and packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
np.random.seed(2018)

# Configure visual settings:
plt.style.use('fivethirtyeight')
sns.set(font_scale=2)
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
## Read Data
df=pd.read_csv('./inputs/cleaned_prescriber_info18.csv')
df.head()[:2]

/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,NPI,Gender,State,Credentials,Specialty,NPPES Provider Last Name,NPPES Provider First Name,NPPES Provider ZIP Code,NPPES Provider State,Totalclaimcount,...,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,avg_op_supply,op_longer,Opioid_Prescriber,op_prescriber
0,1003002320,M,MS,D.M.D. P.A.,Dentist,EKLUND,D.,39208.0,MS,129,...,0,0,0,0,0,0,4.5,1.0,1,1.0
1,1003004771,F,CO,M.D.,Pulmonary Disease,RIGGLE,ERIN,80120.0,CO,537,...,0,0,16,0,0,0,4.5,0.0,0,0.0


In [3]:
print(df.shape)

(25000, 264)


We have a high dimensional data set again. How can we create an effective predictive model quickily on such a complicated data-set?



### Data Proprecessing

Even a quick and dirty classification model requires the transformation of non-numeric feature, in order to enable Python to identify the variable. My method was so 'dirty' that I even included misleading variables in my model. Now I will modify my Kernel to a more completed, accurate one.

In [4]:
opioids=pd.read_csv('./inputs/list_opioids.csv')

In [5]:
name=opioids['Drug Name']
import re
new_name=name.apply(lambda x:re.sub("\ |-",".",str(x)))
columns=df.columns
Abandoned_variables = set(columns).intersection(set(new_name))
Kept_variable=[]
for each in columns:
    if each in Abandoned_variables:
        pass
    else:
        Kept_variable.append(each)

In [6]:
df=df[Kept_variable]
print(df.shape)

(25000, 253)


In [7]:
df.head()[:2]

,NPI,Gender,State,Credentials,Specialty,NPPES Provider Last Name,NPPES Provider First Name,NPPES Provider ZIP Code,NPPES Provider State,Totalclaimcount,...,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,avg_op_supply,op_longer,Opioid_Prescriber,op_prescriber
0,1003002320,M,MS,D.M.D. P.A.,Dentist,EKLUND,D.,39208.0,MS,129,...,0,0,0,0,0,0,4.5,1.0,1,1.0
1,1003004771,F,CO,M.D.,Pulmonary Disease,RIGGLE,ERIN,80120.0,CO,537,...,0,0,16,0,0,0,4.5,0.0,0,0.0


In [8]:
# df['NPPES Provider ZIP Code'] = df['NPPES Provider ZIP Code'].str.replace(',', '')
# df['NPPES Provider ZIP Code'] = df['NPPES Provider ZIP Code'].str.replace(' ', '')
df['NPPES Provider ZIP Code'] = df['NPPES Provider ZIP Code'].astype(float)

In [9]:
df.isnull().sum()

NPI                                  0
Gender                               0
State                                0
Credentials                         82
Specialty                            0
NPPES Provider Last Name          2610
NPPES Provider First Name         2610
NPPES Provider ZIP Code           2610
NPPES Provider State                 0
Totalclaimcount                      0
OpioidClaimCount                     0
ACYCLOVIR                            0
ADVAIR.DISKUS                        0
AGGRENOX                             0
ALENDRONATE.SODIUM                   0
ALLOPURINOL                          0
ALPRAZOLAM                           0
AMIODARONE.HCL                       0
AMITRIPTYLINE.HCL                    0
AMLODIPINE.BESYLATE                  0
AMLODIPINE.BESYLATE.BENAZEPRIL       0
AMOXICILLIN                          0
AMOX.TR.POTASSIUM.CLAVULANATE        0
AMPHETAMINE.SALT.COMBO               0
ATENOLOL                             0
ATORVASTATIN.CALCIUM     

In [10]:
df.columns

Index(['NPI', 'Gender', 'State', 'Credentials', 'Specialty',
       'NPPES Provider Last Name', 'NPPES Provider First Name',
       'NPPES Provider ZIP Code', 'NPPES Provider State', 'Totalclaimcount',
       ...
       'VYTORIN', 'WARFARIN.SODIUM', 'XARELTO', 'ZETIA', 'ZIPRASIDONE.HCL',
       'ZOLPIDEM.TARTRATE', 'avg_op_supply', 'op_longer', 'Opioid_Prescriber',
       'op_prescriber'],
      dtype='object', length=253)

In [11]:
df.drop(['NPPES Provider Last Name', 'NPPES Provider First Name','NPPES Provider ZIP Code',
         'NPPES Provider State'], axis =1, inplace = True)

In [12]:
df.head()[:2]

,NPI,Gender,State,Credentials,Specialty,Totalclaimcount,OpioidClaimCount,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,avg_op_supply,op_longer,Opioid_Prescriber,op_prescriber
0,1003002320,M,MS,D.M.D. P.A.,Dentist,129,29,0,0,0,...,0,0,0,0,0,0,4.5,1.0,1,1.0
1,1003004771,F,CO,M.D.,Pulmonary Disease,537,0,0,61,0,...,0,0,16,0,0,0,4.5,0.0,0,0.0


In [13]:
train,test = train_test_split(df,test_size=0.3,random_state=42)
print(train.shape)
print(test.shape)

(17500, 249)
(7500, 249)


In [14]:
Categorical_columns=['Gender','State','Credentials','Specialty']
for col in Categorical_columns:
    train[col]=pd.factorize(train[col], sort=True)[0]
    test[col] =pd.factorize(test[col],sort=True)[0]

/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
train.iloc[:,1:244][:2]

,Gender,State,Credentials,Specialty,Totalclaimcount,OpioidClaimCount,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VENLAFAXINE.HCL.ER,VENTOLIN.HFA,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL
4913,0,18,516,48,153,36,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9338,0,6,283,25,229,0,0,0,0,0,...,0,0,0,0,0,0,0,0,14,0


In [25]:
features=train.iloc[:,1:245]
features.head()[:2]

,Gender,State,Credentials,Specialty,Totalclaimcount,OpioidClaimCount,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VENTOLIN.HFA,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE
4913,0,18,516,48,153,36,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9338,0,6,283,25,229,0,0,0,0,0,...,0,0,0,0,0,0,0,14,0,0


In [26]:
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier

In [27]:
features=train.iloc[:,1:245]
target = train['Opioid_Prescriber']

In [28]:
Name=[]
Accuracy=[]
model1=LogisticRegression(random_state=22,C=0.000000001,solver='liblinear',max_iter=200)
model2=GaussianNB()
model3=RandomForestClassifier(n_estimators=200,random_state=22)
model4=GradientBoostingClassifier(n_estimators=200)
model5=KNeighborsClassifier()
model6=DecisionTreeClassifier()
model7=LinearDiscriminantAnalysis()
Ensembled_model=VotingClassifier(estimators=[('lr', model1), ('gn', model2), ('rf', model3),('gb',model4),('kn',model5),('dt',model6),('lda',model7)], voting='hard')
for model, label in zip([model1, model2, model3, model4,model5,model6,model7,Ensembled_model], ['Logistic Regression','Naive Bayes','Random Forest', 'Gradient Boosting','KNN','Decision Tree','LDA', 'Ensemble']):
    scores = cross_val_score(model, features, target, cv=5, scoring='accuracy')
    Accuracy.append(scores.mean())
    Name.append(model.__class__.__name__)
    print("Accuracy: %f of model %s" % (scores.mean(),label))

/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.669302 of model Logistic Regression


/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.550866 of model Naive Bayes


/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


...

Accuracy: 0.819438 of model Decision Tree


/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renorma

Accuracy: 0.733602 of model LDA


/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this

Accuracy: 0.865751 of model Ensemble


For the model with the accuracy worse than 80%, we drop them and re_ensemble the model.

In [20]:
Name_2=[]
Accuracy_2=[]
Ensembled_model_2=VotingClassifier(estimators=[('rf', model3),('gb',model4)], voting='hard')
for model, label in zip([model3, model4,Ensembled_model_2], ['Random Forest', 'Gradient Boosting','Ensemble']):
    scores = cross_val_score(model, features, target, cv=5, scoring='accuracy')
    Accuracy_2.append(scores.mean())
    Name_2.append(model.__class__.__name__)
    print("Accuracy: %f of model %s" % (scores.mean(),label))

/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.877110 of model Random Forest


/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.863146 of model Gradient Boosting


/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/prepr

Accuracy: 0.873749 of model Ensemble


We can see that the ensembled model putting two models together performed better than the other three models. I will further use the test set to compare the performances between models.

#### Evaluating with the test set

In [29]:
from sklearn.metrics import accuracy_score
classifers=[model3,model4,Ensembled_model_2]
out_sample_accuracy=[]
Name_2=[]
for each in classifers:
    fit=each.fit(features,target)
    pred=fit.predict(test.iloc[:,1:245])
    accuracy=accuracy_score(test['Opioid_Prescriber'],pred)
    Name_2.append(each.__class__.__name__)
    out_sample_accuracy.append(accuracy)

/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
print('KNN: %f ' %cross_val_score(knn, X_train, y_train).mean())
knn.fit(X_train,y_train)
measure_performance(X_test,y_test,knn)
plot_classifier(X_train,y_train,X_test,y_test,knn)

#### In-sample and out-sample evaluation

In [ ]:
in_sample_accuracy=Accuracy_2

In [ ]:
Index = [1,2,3]
plt.bar(Index,in_sample_accuracy)
plt.xticks(Index, Name_2,rotation=45)
plt.ylabel('Accuracy')
plt.xlabel('Model')
plt.title('In sample accuracy of models')
plt.show()

In [ ]:
plt.bar(Index,out_sample_accuracy)
plt.xticks(Index, Name_2,rotation=45)
plt.ylabel('Accuracy')
plt.xlabel('Model')
plt.title('Out sample accuracies of models')
plt.show()




The hard-voting classifer does not provide a better result in terms of out sample error. Therefore, in this case, the voting classifer cannot reach the satisfied result. A soft voting method can be considered as a substitute.

## Box Plot of Model Performance

In [ ]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Matplotlib  for data visualization
%matplotlib inline
import matplotlib.pyplot as plt

# Seaborn for data visualization
import seaborn as sns

In [ ]:
# Set font size
plt.rcParams.update({'font.size': 15})
# Set figure size
fig = plt.figure(figsize = (10,5))

plt.boxplot(Accuracy, labels = Name)
plt.ylabel('Accuracy')
plt.title('Model performances')

## ROC Curves and AUC of Cross-Validation Prediction

Another way to check the performance or  diagnostic ability of a binary classifier is to plot the receiver operating characteristics (ROC) curves and compute the area under the curve (AUC).

In [ ]:
# Metric measurements
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

################################################################################################
def plot_cv_roc_curve(features, target, cv_fold): 
    
    # Classifiers to be trained 
    gb_clf = GradientBoostingClassifier(random_state = 42)
    forest_clf= RandomForestClassifier(random_state = 42)
    svm_clf = SVC(probability = True)


   # List of the  classifiers
    clf_list = [gb_clf, forest_clf, svm_clf]

    y_scores = []
    for clf in clf_list:
            y_score = cross_val_predict(clf, features, target, cv = cv_fold, 
                                    method ='predict_proba')
            y_scores.append(y_score)
    
    # Assign the predicted probabilities
    y_score_gb =y_scores[0][:,1]
    y_score_forest = y_scores[1][:,1]
    y_score_svm = y_scores[2][:,1]

    # Compute the fpr and tpr for each classifier
    fpr_gb, tpr_gb, thresholds_gb = roc_curve(target, y_score_gb)
    fpr_forest, tpr_forest, thresholds_forest = roc_curve(target, y_score_forest)
    fpr_svm, tpr_svm, thresholds_svm = roc_curve(target, y_score_svm)
    
    # Compute the area under the curve for each classifier
    area_gb =roc_auc_score(training_label, y_score_gb)
    area_forest =roc_auc_score(target, y_score_forest)
    area_svm =roc_auc_score(target, y_score_svm)
    
    # Set figure size
    fig = plt.figure(figsize = (10,8))
    
    # Set font size
    plt.rcParams.update({'font.size': 15})

    # Plotting of the functions
    plt.plot(fpr_gb, tpr_gb, 'r--', label = 'GB (AUC = %0.2f)' % area_gb)
    plt.plot(fpr_forest, tpr_forest, 'b--', label = 'FOREST (AUC = %0.2f)' % area_forest)
    plt.plot(fpr_svm, tpr_svm, 'b-', label = 'SVC (AUC = %0.2f)' % area_svm)
    plt.plot([0,1], [0,1], 'k--')
    
    # Set axis and labels
    plt.axis([0,1,0,1])
    plt.xlabel('False positive rate (FPR)')
    plt.ylabel('True positive rate (TPR)')
    plt.title('ROC Curve of Training Set')
    plt.legend(loc = 'lower right')
 ##########################################################

# Set paramters of the plot function
training_features = X_train_scaled
training_label = y_train_label
cv_fold = 10

# Instantiate the plot function
plot_cv_roc_curve(training_features, training_label, cv_fold)
plt.show()

## Prediction on the Test set

The importance of creating a test set is to check if the machine learning models generalize to an unseen data not used during the training process. Having trained our model on the training set, we are ready to test the models on unseen data -- the test set

In [ ]:
def Test_prediction(training_features, training_label, test_features, test_label):
    
    # Classifiers to be trained 
    gb_clf = GradientBoostingClassifier(random_state = 42)
    forest_clf= RandomForestClassifier(random_state = 42)
    svm_clf = SVC()
 
   # List of the  classifiers
    clf_list = [gb_clf, forest_clf, svm_clf]

    # Empty lists to append the result of each classifier
    scores = []
    confmaxs = []
    clf_reps= []
    
    for clf in clf_list:
        
        clf.fit(training_features, training_label)
        y_pred = clf.predict(test_features)
        score = accuracy_score(test_label, y_pred)
        confmax = confusion_matrix(test_label, y_pred)
        clf_rep = classification_report(test_label, y_pred)
        
        scores.append(score)
        confmaxs.append(confmax)
        clf_reps.append(clf_rep)
    
    return  scores, confmaxs, clf_reps

# Set paramters of the function
training_features = X_train_scaled
training_label = y_train_label
test_features = X_test_scaled
test_label = y_test_label

# Instantiate the function
predictions = Test_prediction(training_features, training_label, test_features, test_label)

# Get the values of the function
test_scores = predictions[0]
test_confmax = predictions[1]
test_clf_reps =  predictions[2]

In [ ]:
#  Display the test score in a dataframe
pd.DataFrame({'Models': ['GB', 'Forest', 'SVC'], 'Test Accuracy': test_scores})

The test accuracy shows that the models generalize very well to unseen data

In [ ]:
# Confusion Matrix and Classification Report of Gradient Boosting
print('Confusion Matrix for GB:\n', test_confmax[0])
print('\n')
print('Classification Report for GB:\n', test_clf_reps[0])

In [ ]:
# Confusion Matrix and Classification Report of Random Forest
print('Confusion Matrix for Forest:\n', test_confmax[1])
print('\n')
print('Classification Report for Forest:\n', test_clf_reps[1])

In [ ]:
# Confusion Matrix and Classification Report of Support Vector Machine
print('Confusion Matrix for SVM: \n', test_confmax[2])
print('\n')
print('Classification Report for SVM: \n', test_clf_reps[2])

## ROC Curves and AUC of Test Set

In [ ]:
# Metric measurements
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

###############################################################################################################
def plot_test_roc_curve(training_features, training_label, test_features, test_label): 
    
    # Classifiers to be trained 
    gb_clf = GradientBoostingClassifier(random_state = 42)
    forest_clf= RandomForestClassifier(random_state = 42)
    svm_clf = SVC(probability = True)
 
   # List of the  classifiers
    clf_list = [gb_clf, forest_clf, svm_clf]

    y_proba = []
    for clf in clf_list:
        clf.fit(training_features, training_label)
        pred_proba = clf.predict_proba(test_features)
        y_proba.append(pred_proba)
    
    # Assign the predicted probabilities
    y_proba_gb = y_proba[0][:, 1]
    y_proba_forest = y_proba[1][:, 1]
    y_proba_svm = y_proba[2][:, 1]
    
    # Compute the fpr and tpr for each classifier
    fpr_gb, tpr_gb, thresholds_gb = roc_curve(test_label, y_proba_gb)
    fpr_forest, tpr_forest, thresholds_forest = roc_curve(test_label, y_proba_forest)
    fpr_svm, tpr_svm, thresholds_svm = roc_curve(test_label, y_proba_svm)
    
    # Compute the area under the curve for each classifier
    area_gb =roc_auc_score(test_label, y_proba_gb)
    area_forest =roc_auc_score(test_label, y_proba_forest)
    area_svm =roc_auc_score(test_label, y_proba_svm)
    
    # Set figure size
    fig = plt.figure(figsize = (10,8))
    
    # Set font size
    plt.rcParams.update({'font.size': 15})

    # Plotting of the functions
    plt.plot(fpr_gb, tpr_gb, 'r--', label = 'GB (AUC = %0.2f)' % area_gb)
    plt.plot(fpr_forest, tpr_forest, 'b-', label = 'FOREST (AUC = %0.2f)' % area_forest)
    plt.plot(fpr_svm, tpr_svm, 'b--', label = 'SVC (AUC = %0.2f)' % area_svm)
    plt.plot([0,1], [0,1], 'k--')
    
    # Set axis and labels
    plt.axis([0,1,0,1])
    plt.xlabel('False positive rate (FPR)')
    plt.ylabel('True positive rate (TPR)')
    plt.title('ROC Curve of Test Set')
    plt.legend(loc = 'lower right')
 ####################################################################################################################

# Set paramters of the function
training_features = X_train_scaled
training_label = y_train_label
test_features = X_test_scaled
test_label = y_test_label

# Instantiate the plot function
plot_test_roc_curve(training_features, training_label, test_features, test_label)
plt.show()